# Account Sharing Detection

This notebook shows how to detect when multiple people share a single account (e.g., Netflix password sharing).

**Key Insight**: A single person has consistent preferences. Multiple people using one account will show inconsistent, cyclic preferences.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyrevealed import ConsumerSession, compute_aei, compute_mpi, check_garp

## 1. Simulating Single vs Shared Accounts

For a streaming service:
- **Price** = Time cost (opportunity cost of watching)
- **Quantity** = Hours watched per genre

In [ ]:
def generate_single_user_profile(n_days=30, preference_weights=None):
    """
    Generate viewing data for a single user with consistent preferences.
    
    Genres: [Action, Romance, Comedy, Documentary]
    """
    n_genres = 4
    
    if preference_weights is None:
        # Random but consistent preference weights
        preference_weights = np.random.dirichlet(np.ones(n_genres) * 2)
    
    # Time costs vary by day (e.g., busier on weekdays)
    prices = np.random.uniform(0.5, 2.0, (n_days, n_genres))
    
    # Viewing hours based on preferences and available time
    total_time = np.random.uniform(2, 6, n_days)  # 2-6 hours per day
    
    quantities = np.zeros((n_days, n_genres))
    for day in range(n_days):
        # Adjust for price: watch more of what's "cheap" (easy to watch)
        adjusted_prefs = preference_weights / prices[day]
        adjusted_prefs = adjusted_prefs / adjusted_prefs.sum()
        quantities[day] = adjusted_prefs * total_time[day]
        
        # Add small noise
        quantities[day] += np.random.normal(0, 0.1, n_genres)
        quantities[day] = np.maximum(quantities[day], 0)
    
    return ConsumerSession(prices=prices, quantities=quantities)


def generate_shared_account(n_days=30):
    """
    Generate viewing data for a shared account (2 different users).
    
    User A: Prefers Action, Documentary
    User B: Prefers Romance, Comedy
    """
    n_genres = 4
    
    # Two very different preference profiles
    user_a_prefs = np.array([0.5, 0.05, 0.05, 0.4])  # Action + Documentary
    user_b_prefs = np.array([0.05, 0.5, 0.4, 0.05])  # Romance + Comedy
    
    prices = np.random.uniform(0.5, 2.0, (n_days, n_genres))
    quantities = np.zeros((n_days, n_genres))
    
    for day in range(n_days):
        # Randomly choose which user watches (simulate alternating usage)
        if np.random.random() < 0.5:
            prefs = user_a_prefs
        else:
            prefs = user_b_prefs
        
        total_time = np.random.uniform(2, 6)
        adjusted_prefs = prefs / prices[day]
        adjusted_prefs = adjusted_prefs / adjusted_prefs.sum()
        quantities[day] = adjusted_prefs * total_time
        quantities[day] = np.maximum(quantities[day] + np.random.normal(0, 0.1, n_genres), 0)
    
    return ConsumerSession(prices=prices, quantities=quantities)

In [ ]:
np.random.seed(42)

# Generate profiles
single_user = generate_single_user_profile(n_days=40)
shared_account = generate_shared_account(n_days=40)

print("Generated 40 days of viewing data for:")
print("  - Single user account")
print("  - Shared account (2 users)")

## 2. Analyzing Consistency

In [ ]:
# Compute metrics
single_aei = compute_aei(single_user)
shared_aei = compute_aei(shared_account)

single_mpi = compute_mpi(single_user)
shared_mpi = compute_mpi(shared_account)

print("Consistency Analysis:")
print("\nSingle User Account:")
print(f"  AEI Score: {single_aei.efficiency_index:.4f}")
print(f"  MPI Score: {single_mpi.mpi_value:.4f}")
print(f"  Is consistent: {single_aei.is_perfectly_consistent}")

print("\nShared Account:")
print(f"  AEI Score: {shared_aei.efficiency_index:.4f}")
print(f"  MPI Score: {shared_mpi.mpi_value:.4f}")
print(f"  Is consistent: {shared_aei.is_perfectly_consistent}")

## 3. Rolling Window Analysis

Detect when an account starts being shared by monitoring consistency over time.

In [ ]:
def analyze_consistency_over_time(session, window_size=10):
    """
    Calculate AEI in rolling windows to detect changes.
    """
    scores = []
    windows = session.split_by_window(window_size)
    
    for i, window in enumerate(windows):
        aei = compute_aei(window)
        scores.append({
            'window': i,
            'start_day': i * window_size,
            'aei': aei.efficiency_index
        })
    
    return scores


# Analyze over time
single_over_time = analyze_consistency_over_time(single_user, window_size=8)
shared_over_time = analyze_consistency_over_time(shared_account, window_size=8)

# Plot
fig, ax = plt.subplots(figsize=(10, 5))

single_scores = [s['aei'] for s in single_over_time]
shared_scores = [s['aei'] for s in shared_over_time]

ax.plot(single_scores, 'b-o', label='Single User', markersize=8)
ax.plot(shared_scores, 'r-o', label='Shared Account', markersize=8)
ax.axhline(0.90, color='orange', linestyle='--', label='Alert Threshold')

ax.set_xlabel('Time Window')
ax.set_ylabel('AEI Score')
ax.set_title('Consistency Score Over Time')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1.05)

plt.tight_layout()
plt.show()

## 4. Detection Pipeline

In [ ]:
def detect_account_sharing(session, aei_threshold=0.90, window_size=10):
    """
    Detect potential account sharing.
    
    Returns:
        is_shared: Boolean indicating likely sharing
        overall_aei: Overall consistency score
        variance: Variance in rolling AEI (high variance = suspicious)
        recommendation: Action to take
    """
    overall = compute_aei(session)
    
    # Rolling analysis
    windows = analyze_consistency_over_time(session, window_size)
    window_scores = [w['aei'] for w in windows]
    
    variance = np.var(window_scores) if len(window_scores) > 1 else 0
    
    # Decision logic
    is_shared = overall.efficiency_index < aei_threshold or variance > 0.02
    
    if is_shared:
        if overall.efficiency_index < 0.7:
            recommendation = "REQUIRE_2FA: Very likely shared account"
        elif overall.efficiency_index < 0.85:
            recommendation = "SUGGEST_PROFILE: Recommend creating separate profiles"
        else:
            recommendation = "MONITOR: Minor inconsistency, continue monitoring"
    else:
        recommendation = "NORMAL: Account appears to have single user"
    
    return {
        'is_shared': is_shared,
        'overall_aei': overall.efficiency_index,
        'variance': variance,
        'recommendation': recommendation
    }


# Test detection
single_result = detect_account_sharing(single_user)
shared_result = detect_account_sharing(shared_account)

print("Detection Results:")
print("\nSingle User Account:")
for k, v in single_result.items():
    print(f"  {k}: {v}")

print("\nShared Account:")
for k, v in shared_result.items():
    print(f"  {k}: {v}")

## Key Takeaways

1. **Single users have consistent preferences** - high AEI scores

2. **Shared accounts show "impossible" preference patterns** - one day they love action, next day they only watch romance

3. **Rolling window analysis** helps detect when sharing begins

4. **This is a behavioral signal**, not IP-based - harder to circumvent